In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os

from vigor import predicates
from vigor.evaluation import *

C:\Users\brian\anaconda3\envs\vigor\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


device cpu


# Generate Graphs and Statistics

1. Generate graphs using the fast_gnp_random_graph function from networkx

2. Calculate statistics for each graph

    * s_metric, sigma, and is_planar are prohibitively expensive to compute for a large number of graphs

In [3]:
file_path = 'graph_data'
if not os.path.exists(f'{file_path}.csv'):
    print('generating graph data')
    df = generate_graphs(300, 2, 200)
else:
    print('loading graph data')
    df = pd.read_csv(f'{file_path}.csv')

loading graph data


In [4]:
df

,graph_type,is_directed_int,has_spatial_attributes,has_temporal_attributes,is_planar,is_bipartite,n_components,avg_betweenness_centrality,avg_closeness_centrality,avg_eigenvector_centrality,...,sigma,n_nodes,node_types,node_attributes,number_of_isolates,density,edge_types,edge_attributes,n_parallel_edges,n_self_loops
0,4,0,0,0,NaN,0,1,0.000534,0.947804,0.097103,...,NaN,106,0,0.0,0,0.944474,0,0.0,0,0
1,4,0,0,0,NaN,0,1,0.004174,0.583721,0.075489,...,NaN,173,0,0.0,0,0.286194,0,0.0,0,0
2,4,0,0,0,NaN,0,1,0.004833,0.641326,0.091558,...,NaN,118,0,0.0,0,0.439374,0,0.0,0,0
3,4,0,0,0,NaN,0,2,0.102564,0.376141,0.240887,...,NaN,13,0,0.0,1,0.179487,0,0.0,0,0
4,4,0,0,0,NaN,0,1,0.000759,0.947525,0.114659,...,NaN,76,0,0.0,0,0.943860,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,4,0,0,0,NaN,0,1,0.001242,0.856071,0.085069,...,NaN,138,0,0.0,0,0.831059,0,0.0,0,0
293,3,0,0,0,NaN,0,8,0.050950,0.140664,0.065252,...,NaN,72,0,0.0,6,0.029343,0,0.0,0,0
294,4,0,0,0,NaN,0,1,0.004470,0.575750,0.076719,...,NaN,167,0,0.0,0,0.262463,0,0.0,0,0
295,4,0,0,0,NaN,0,1,0.001171,0.815435,0.071373,...,NaN,196,0,0.0,0,0.772894,0,0.0,0,0


# Prepare Data for Predicate Learning

In [5]:
attr_std = df.std()

In [6]:
attr_std

graph_type                     0.396574
is_directed_int                0.000000
has_spatial_attributes         0.000000
has_temporal_attributes        0.000000
is_planar                           NaN
is_bipartite                   0.189171
n_components                   1.400327
avg_betweenness_centrality     0.029138
avg_closeness_centrality       0.190156
avg_eigenvector_centrality     0.091201
avg_degree                    40.999756
std_degree                     1.722291
clustering_coefficient         0.295832
transitivity                   0.295636
modularity                     0.104869
communities                    1.401977
avg_shortest_path_length       0.699630
radius                         0.856768
diameter                       1.055290
assortativity                  0.112348
vertex_connectivity           38.332860
eccentricity_avg               0.922103
s_metric                            NaN
sigma                               NaN
n_nodes                       56.939827


In [7]:
attr_null = df.isnull().any(axis=0)

In [8]:
attr_null

graph_type                    False
is_directed_int               False
has_spatial_attributes        False
has_temporal_attributes       False
is_planar                      True
is_bipartite                  False
n_components                  False
avg_betweenness_centrality    False
avg_closeness_centrality      False
avg_eigenvector_centrality    False
avg_degree                    False
std_degree                    False
clustering_coefficient        False
transitivity                  False
modularity                    False
communities                    True
avg_shortest_path_length      False
radius                        False
diameter                      False
assortativity                  True
vertex_connectivity           False
eccentricity_avg              False
s_metric                       True
sigma                          True
n_nodes                       False
node_types                    False
node_attributes               False
number_of_isolates          

**Only use attributes that do not contain null values and have a standard deviation > 0:**

In [9]:
attr_use = (attr_std.fillna(0)>0) & ~attr_null

In [10]:
attr_use

graph_type                     True
is_directed_int               False
has_spatial_attributes        False
has_temporal_attributes       False
is_planar                     False
is_bipartite                   True
n_components                   True
avg_betweenness_centrality     True
avg_closeness_centrality       True
avg_eigenvector_centrality     True
avg_degree                     True
std_degree                     True
clustering_coefficient         True
transitivity                   True
modularity                     True
communities                   False
avg_shortest_path_length       True
radius                         True
diameter                       True
assortativity                 False
vertex_connectivity            True
eccentricity_avg               True
s_metric                      False
sigma                         False
n_nodes                        True
node_types                    False
node_attributes               False
number_of_isolates          

**Exclude the nominal attribute `graph_type`**:

In [100]:
attr_use['graph_type'] = False

**Excluded Attributes:**

In [101]:
attr_use[~attr_use].index

Index(['graph_type', 'is_directed_int', 'has_spatial_attributes',
       'has_temporal_attributes', 'is_planar', 'communities', 'assortativity',
       's_metric', 'sigma', 'node_types', 'node_attributes', 'edge_types',
       'edge_attributes', 'n_parallel_edges', 'n_self_loops'],
      dtype='object')

In [102]:
attr = attr_use[attr_use].index

In [103]:
attr

Index(['is_bipartite', 'n_components', 'avg_betweenness_centrality',
       'avg_closeness_centrality', 'avg_eigenvector_centrality', 'avg_degree',
       'std_degree', 'clustering_coefficient', 'transitivity', 'modularity',
       'avg_shortest_path_length', 'radius', 'diameter', 'vertex_connectivity',
       'eccentricity_avg', 'n_nodes', 'number_of_isolates', 'density'],
      dtype='object')

In [104]:
len(attr)

18

# Load Expert Predicates

In [105]:
predicates_labels = get_predicate_labels(df, predicates)

**List of predicates for each vis type:**

In [106]:
predicates_labels['predicates']

{'NODELINK': [{'density': [0, 0.1]},
  {'avg_degree': [1, 3]},
  {'clustering_coefficient': [0.1, 0.4]},
  {'node_types': [1, 3]},
  {'edge_types': [1, 2]}],
 'MATRIX': [{'density': [0.1, 1]},
  {'avg_degree': [10, 50]},
  {'modularity': [0.3, 0.7]},
  {'node_attributes': [2, 10]},
  {'edge_attributes': [1, 5]}],
 'NODETRIX': [{'communities': [4, 10]},
  {'clustering_coefficient': [0.5, 1]},
  {'density': [0.1, 0.5]},
  {'node_types': [2, 5]},
  {'modularity': [0.3, 0.8]},
  {'avg_degree': [5, 15]},
  {'node_attributes': [3, 10]},
  {'edge_types': [1, 3]}],
 'NODELINK_MAP': [{'avg_degree': [1, 5]}],
 'PAOHVIS': [{'n_nodes': [50, 500]},
  {'node_types': [3, 6]},
  {'edge_types': [2, 5]},
  {'density': [0.05, 0.2]},
  {'avg_degree': [5, 10]},
  {'transitivity': [0.2, 0.6]}],
 'CHORD_DIAGRAM': [{'n_nodes': [0, 6]},
  {'edge_types': [1, 3]},
  {'clustering_coefficient': [0.3, 0.7]},
  {'avg_degree': [2, 4]}],
 'TREEMAP': [{'graph_type': [0.5, 1.5]},
  {'modularity': [0.5, 1]},
  {'n_nodes'

**Vis type label for each graph:**

In [107]:
predicates_labels['labels']

0        MATRIX
1        MATRIX
2       PAOHVIS
3      NODELINK
4        MATRIX
         ...   
292      MATRIX
293    NODELINK
294      MATRIX
295      MATRIX
296      MATRIX
Length: 297, dtype: object

For each graph, each vis type gets one point for a matching predicate. The vis type with the most total points/matching predicates is chosen as the label.

# Learn Predicates

In [108]:
learned_predicates = get_predicates_labels(df, attr_, predicates_labels['labels'], n_iter=1000)

[   0] loss 2.674076557159424
[ 100] loss 1.0870511531829834
[ 200] loss 0.8246506452560425
[ 300] loss 0.8227745890617371
[ 400] loss 0.8139647841453552
[ 500] loss 0.8101377487182617
[ 600] loss 0.8114305734634399
[ 700] loss 0.8147532939910889
[ 800] loss 0.8184369802474976
[ 900] loss 0.8213256597518921

brush = 0
accuracy = 0.8282828282828283
precision = 0.8921568627450981
recall = 0.8625592417061612
f1 = 0.8771084337349399

[   0] loss 3.294844627380371
[ 100] loss 1.7169829607009888
[ 200] loss 1.3372739553451538
[ 300] loss 1.264787197113037
[ 400] loss 1.2419618368148804
[ 500] loss 1.2306944131851196
[ 600] loss 1.223823070526123
[ 700] loss 1.2191402912139893
[ 800] loss 1.2157268524169922
[ 900] loss 1.213122844696045

brush = 0
accuracy = 0.6666666666666666
precision = 0.4624277456647399
recall = 0.9302325581395349
f1 = 0.6177606177606177

[   0] loss 2.782708168029785
[ 100] loss 1.296474575996399
[ 200] loss 1.0161536931991577
[ 300] loss 0.912499725818634
[ 400] loss 0.

In [93]:
learned_predicates.keys()

dict_keys(['MATRIX', 'PAOHVIS', 'NODELINK', 'NODETRIX', 'CHORD_DIAGRAM'])

In [96]:
for key in learned_predicates.keys():
    try:
        print(key)
        pd.Series(learned_predicates[key][0][0]['params'][0][0].detach().numpy()).isnull()
    except:
        pass

MATRIX
PAOHVIS
NODELINK
NODETRIX
CHORD_DIAGRAM


In [111]:
attr[[0, 1, -2]]

Index(['is_bipartite', 'n_components', 'number_of_isolates'], dtype='object')

In [109]:
for vistype, predicate in learned_predicates.items():
    print(vistype)
    print(predicate)

MATRIX
({'n_components': [0.0, 0.10526315789473684], 'avg_betweenness_centrality': [0.0, 0.1338050328195095], 'avg_eigenvector_centrality': [0.00829293260219581, 0.42838944494724274], 'std_degree': [0.03997473384403583, 0.7610229551792145], 'diameter': [0.10262876749038696, 0.5672981142997742], 'number_of_isolates': [0.0, 0.18181818181818182]}, {'clustering_coefficient': [0.0, 0.6284277439117432]})
PAOHVIS
({}, {'std_degree': [0.0, 0.6946551203727722], 'diameter': [0.0, 0.5875054746866226]})
NODELINK
({'avg_degree': [0.003195298575763901, 0.02762993474337021], 'vertex_connectivity': [0.0, 0.011834319526627219]}, {'n_components': [0.0, 0.10526315789473684], 'avg_betweenness_centrality': [0.0, 0.15302516520023346], 'eccentricity_avg': [0.23331156373023987, 0.7999769151210785], 'number_of_isolates': [0.0, 0.18181818181818182]})
NODETRIX
({0: {'params': (tensor([[    nan,     nan,  0.5385, -0.0594,  0.9583, -0.9628, -1.0104, -0.0983,
         -0.1269, -0.0706,  0.3192, -0.0260,  0.4805, -0